In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Sample Runing Just Ignore this

In [ ]:
df_market = pd.read_csv('../input/market-data/Market_Basket_Optimisation.csv', header=None)

In [ ]:
df_market

In [ ]:
# replacing empty value with 0.
df_market.fillna(0,inplace=True)

In [ ]:
# Data Pre-processing step

# for using aprori , need to convert data in list format..
# transaction = [['apple','almonds'],['apple'],['banana','apple']]....

transactions = []

for i in range(0,len(df_market)):
    transactions.append([str(df_market.values[i,j]) for j in range(0,20) if str(df_market.values[i,j])!='0'])

In [ ]:
transactions[0]

In [ ]:
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

In [ ]:
def perform_rule_calculation(transact_items_matrix, rule_type="fpgrowth", min_support=0.001):
    
    start_time = 0
    total_execution = 0
    
    if(not rule_type=="fpgrowth"):
        start_time = time.time()
        rule_items = apriori(transact_items_matrix, 
                       min_support=min_support, 
                       use_colnames=True, low_memory=True)
        total_execution = time.time() - start_time
        print("Computed Apriori!")
    else:
        start_time = time.time()
        rule_items = fpgrowth(transact_items_matrix, 
                       min_support=min_support, 
                       use_colnames=True)
        total_execution = time.time() - start_time
        print("Computed Fp Growth!")
    
    return total_execution

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
import time
n_range = range(2, 20, 1)
list_time_ap = []
list_time_fp = []
for n in n_range:
    time_ap = 0
    time_fp = 0
    min_sup = float(n/100)
    time_ap = perform_rule_calculation(df, rule_type="fpgrowth", min_support=min_sup)
    time_fp = perform_rule_calculation(df, rule_type="aprior", min_support=min_sup)
    list_time_ap.append(time_ap)
    list_time_fp.append(time_fp)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(n_range, list_time_ap, label='Apriori', color='green')
plt.plot(n_range, list_time_fp, label='Fp_growth', color='red')
plt.xlabel("Support (%)")
plt.ylabel("Run Time (seconds)")
plt.legend(loc="best")
plt.show()

# Main Runing

In [ ]:
#read_data
df = pd.read_csv('../input/rule-sub-datacsv/Online Retail (1).csv')

In [ ]:
#data_preprocessing
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df.dropna(axis=0, subset=['Description'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]
df = df[~df['InvoiceNo'].str.contains('A')]

In [ ]:
#remove row has quantity <= 0
df = df[df.Quantity > 0]

In [ ]:
df

In [ ]:
def check_lowercase(string):
    if string.isupper():
        return True
    return False

In [ ]:
#when we analysis we see that items has decripstion lowercase is not important for ARM
#We need to drop that rows
df.drop(df[df['Description'].str.isupper() == False].index, inplace=True)

In [ ]:
# data latest
df

In [ ]:
#we just need two columns for this problem
sub_df = df[['InvoiceNo', 'Description']]

In [ ]:
sub_df

**Here we have data with each row is single items with invoice number, we need to group all items has same invoice number to make is become a transaction**

In [ ]:
#group row has same invoiceNo
df_trans = sub_df.groupby('InvoiceNo')['Description'].apply(list)

In [ ]:
df_trans

**To can use mlxtend for rule mining, we must convert data to requirement form of mlxtend**

In [ ]:
# we use mlxtend tools for rule mining
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder() # convert df_trans to transaction encoder
te_ary = te.fit(df_trans).transform(df_trans)
df_s = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
# each column corresponding to each item we have in this dataset
df_s

**Now we need funtion two compute runtime of apriori and fp_growth algorithm**

In [ ]:
#function to compute runtime algorithms
def perform_rule_calculation(transact_items_matrix, rule_type="fpgrowth", min_support=0.001):
    
    start_time = 0
    total_execution = 0
    
    if(not rule_type=="fpgrowth"):
        start_time = time.time()
        rule_items = apriori(transact_items_matrix, 
                       min_support=min_support, 
                       use_colnames=True, low_memory=True) # find frequent itemsets with apriori
        total_execution = time.time() - start_time
        print("Computed Apriori!")
    else:
        start_time = time.time()
        rule_items = fpgrowth(transact_items_matrix, 
                       min_support=min_support, 
                       use_colnames=True) #find frequent itemsets with fpgrowth
        total_execution = time.time() - start_time
        print("Computed Fp Growth!")
    
    return total_execution

In [ ]:
#compute runtime 2 algorithms
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
import time
n_range = range(1, 10, 1)
list_time_ap = [] # list runtime of apriori
list_time_fp = [] # list runtime of fp_growth
for n in n_range:
    time_ap = 0
    time_fp = 0
    min_sup = float(n/100)
    time_ap = perform_rule_calculation(df_s, rule_type="fpgrowth", min_support=min_sup)
    time_fp = perform_rule_calculation(df_s, rule_type="aprior", min_support=min_sup)
    list_time_ap.append(time_ap)
    list_time_fp.append(time_fp)

**Plot RunTime Apriori, Fp_growth Algorithm with different min_sup and compare them**

In [ ]:
import matplotlib.pyplot as plt

plt.plot(n_range, list_time_ap, label='Apriori', color='green')
plt.plot(n_range, list_time_fp, label='Fp_growth', color='red')
plt.xlabel("Support (%)")
plt.ylabel("Run Time (seconds)")
plt.legend(loc="best")
plt.show()

> We can see here, fp_growth run significantly faster than apriori with small min_sup.
> When we increase min_sup to 5(%) and bigger apriori has the same runtime with fp_growth

In [ ]:
#save data
sub_df.to_csv('sub_data_clean.csv', index=False)

# EDA DATASET

In [ ]:
#plot top 10 items most appearance
sub_df['Description'].value_counts()[:20].plot(kind='bar')

**Dataset has 19847 Transaction and 3987 Items**

In [ ]:
df_s.shape